In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
from itertools import chain, combinations

tf.compat.v1.disable_eager_execution()

In [2]:
# variables
num_epochs = 40000
val_loss_min_improvement = 0.0001
val_loss_not_improved_for = 10
min_val_loss = 0.2
num_successes = 10  
experiment = {'LR': [0.1, 0.01], 'hidden': [2, 4], 'bridge': [True, False]}


# Dataset
x_train = [[0, 0], [0, 1], [1, 0], [1, 1]]
t_train = [[0], [1], [1], [0]]
x_val = [[0, 0], [0, 1], [1, 0], [1, 1], [1, 0.1], [1, 0.9], [0.9, 0.9], [0.1, 0.9]]
t_val = [[0], [1], [1], [0], [1], [0], [0], [1]]

In [3]:
# print hyper parameters to file:
data_to_write = "Hyper Paramwters:\n"
data_to_write += "Max epocs=" + str(num_epochs) + "\n"
data_to_write += "stop_Min_loss<" + str(min_val_loss) + " and loss_Not_Improve_for " + str(val_loss_not_improved_for) + " epocs Loss_Min_adv=" + str(val_loss_min_improvement) + "\n\n"
data_to_write += """x_train = [[0, 0], [0, 1], [1, 0], [1, 1]]
t_train = [[0], [1], [1], [0]]
x_val = [[0, 0], [0, 1], [1, 0], [1, 1], [1, 0.1], [1, 0.9], [0.9, 0.9], [0.1, 0.9]]
t_val = [[0], [1], [1], [0], [1], [0], [0], [1]]"""
open("output.txt", "a").write(data_to_write)

314

In [4]:
def powerset(iterable):
    """
    Given an iterable, finds all its subsets 
    powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)
    """
    s = list(iterable)
    ps = chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
    ps = [set(a) for a in ps]
    return ps

In [5]:

def create_experiments():
    """
    Returns a list of all possible experiment parameters sets.
    """
    global experiment

    key_set = set(experiment.keys())
    ps = list(powerset(key_set))
    experiment_lst = list()

    for s in ps:
        complement = key_set - s
        params = dict()

        for key in s:
            params[key] = experiment[key][1]

        for key in complement:
            params[key] = experiment[key][0]

        experiment_lst.append(params)
    
    # Add the last experiments (bridge=True and hidden=1)
    experiment_lst += [{ 'LR': 0.1, 'hidden': 1, 'bridge': True }, { 'LR': 0.01, 'hidden': 1, 'bridge': True }]
    return experiment_lst

In [6]:
experiments = create_experiments()
experiments

[{'LR': 0.1, 'bridge': True, 'hidden': 2},
 {'LR': 0.01, 'bridge': True, 'hidden': 2},
 {'LR': 0.1, 'bridge': False, 'hidden': 2},
 {'LR': 0.1, 'bridge': True, 'hidden': 4},
 {'LR': 0.01, 'bridge': False, 'hidden': 2},
 {'LR': 0.01, 'bridge': True, 'hidden': 4},
 {'LR': 0.1, 'bridge': False, 'hidden': 4},
 {'LR': 0.01, 'bridge': False, 'hidden': 4},
 {'LR': 0.1, 'bridge': True, 'hidden': 1},
 {'LR': 0.01, 'bridge': True, 'hidden': 1}]

In [7]:
def init_results_dict():
    results = dict()
    results["meanepochs"] = 0
    results["std/epochs%"] = 0
    results["Failures"] = 0
    results["meanvalidloss"] = 0
    results["stdvalidlossPercent"] = 0
    results["meanTrainLoss"] = 0
    results["stdTrainLossPercent"] = 0
    results["hidden_node_outputs"] = None
    return results

In [8]:
results = init_results_dict()
results

{'Failures': 0,
 'hidden_node_outputs': None,
 'meanTrainLoss': 0,
 'meanepochs': 0,
 'meanvalidloss': 0,
 'std/epochs%': 0,
 'stdTrainLossPercent': 0,
 'stdvalidlossPercent': 0}

In [9]:
def rand_weights(in_dim, h_dim, out_dim, is_concat=False):
    """
    Generates random weights for a neural network
    """
    w1 = tf.Variable(tf.random.uniform([in_dim, h_dim], -1, 1))
    w2 = tf.Variable(tf.random.uniform([h_dim+in_dim, out_dim], -1, 1)) \
        if is_concat else tf.Variable(tf.random.uniform([h_dim, out_dim], -1, 1))

    b1 = tf.Variable(tf.zeros([h_dim]))
    b2 = tf.Variable(tf.zeros([out_dim]))

    return w1, w2, b1, b2

In [10]:
def linear_model(x, w1, w2, b1, b2, is_bridge=False, t=0.55):
    """
    Given weights, biases and an input x,
    Returns the layers of a linear model
    """

    z1 = tf.matmul(x, w1) + b1
    hlayer = tf.sigmoid(z1 / t)

    if not is_bridge:
        z2 = tf.matmul(hlayer,w2) + b2
    else:
        hlayer_concat = tf.concat([hlayer,x], 1)
        z2 = tf.matmul(hlayer_concat,w2) + b2

    olayer = tf.sigmoid(z2/t)
    return olayer, hlayer

In [11]:
def train_GD(x_train, t_train, x_val, t_val, experiment_params, prev_results):
    """
    Training function
    @param x_train: The training set of samples (list)
    @param t_train: The training set labels (list)
    @param x_val: The validation set of samples (list)
    @param t_val: The validation set labels (list)
    @param experiment_params: The current experiment parameters (dict)
    @param prev_results: The previous training results, e.g. mean values, etc. (dict) 

    Returns the experiment results (dict)
    """

    global num_epochs, num_successes, val_loss_min_improvement, val_loss_not_improved_for, min_val_loss

    in_dim = len(x_train[0]) # Input dimension num of columns in (truth-table)
    out_dim = 1
    h_dim = experiment_params["hidden"]
    bridge = experiment_params["bridge"]
    lr = experiment_params["LR"]

    # Placeholders
    x = tf.compat.v1.placeholder(tf.float32, [None, in_dim])
    t = tf.compat.v1.placeholder(tf.float32, [None, out_dim])

    # Initialize weights and biases
    w1, w2, b1, b2 = rand_weights(in_dim, h_dim, out_dim, bridge)

    # Model definition
    output_node, hlayer = linear_model(x, w1, w2, b1, b2, bridge)

    # CE loss definition
    CE = -1 * (tf.math.reduce_sum(t * tf.math.log(output_node) + (tf.ones([1])-t) * tf.math.log(1-output_node)))

    # Optimizer definition
    optimizer = tf.compat.v1.train.GradientDescentOptimizer(lr)
    train = optimizer.minimize(CE)

    # Session records
    epoch_lst, h_node_lst = list(), list()
    train_loss_lst, val_loss_lst = list(), list()
    num_failures, successes_count = 0, 0
    new_w1, new_w2, new_b1, new_b2 = w1, w2, b1, b2

    while successes_count < num_successes:
        print(f"Training Session NO'{num_failures+successes_count}...")
        loss_t, stop_epoch = 0, 0
        loss_v, prev_v_loss = 0, 0
        loss_val_list = []

        w1_var = tf.compat.v1.assign(w1, new_w1)
        w2_var = tf.compat.v1.assign(w2, new_w2)
        b1_var = tf.compat.v1.assign(b1, new_b1)
        b2_var = tf.compat.v1.assign(b2, new_b2)

        # Define a session initializer
        init = tf.compat.v1.global_variables_initializer()

        # Start a training session
        with tf.compat.v1.Session() as sess:
            sess.run(init)
            sess.run([w1_var, w2_var, b1_var, b2_var])

            for e in range(num_epochs):

                # Training step
                sess.run(train, {x: x_train, t: t_train})
                
                # Save losses
                loss_v = sess.run(CE, {x: x_val, t: t_val})
                loss_val_list.append(loss_v)
                loss_t = sess.run(CE, {x: x_train, t: t_train})

                # Stop conditions
                stop_condition1 = abs(prev_v_loss-loss_v) < val_loss_min_improvement # and e+1 > num_epochs-10
                stop_condition2 = loss_v < min_val_loss

                # print for check:
                """
                if e % 500 == 0 :
                  print("prev_v_loss = ", prev_v_loss)
                  print("loss_v = ", loss_v)
                  print(stop_condition1, stop_condition2)
                  print(f"Epoch-{e+1}: train loss: {loss_t}, validation loss: {loss_v}")
                """

                if stop_condition1 and stop_condition2:
                  stop_epoch = e+1
                  break

                # print for check:
                """"
                if e < 100 :
                  print("+++loss_val_list = ", loss_val_list)
                  print("prev_v_loss = ", prev_v_loss)
                  print("loss_v = ", loss_v)
                """

                # save the validation-loss value, before 10 epochs
                if e+1 >= 10:
                  prev_v_loss = loss_val_list[0]
                  loss_val_list = loss_val_list[1:]

                # check and count failure
                if e+1 == num_epochs:
                    num_failures += 1
                    stop_epoch = num_epochs
                    print("fail")

            # Save the output of the hidden node
            h_node_lst += [sess.run(hlayer, {x: x_val})]

        # Save successful experiments
        if stop_epoch < num_epochs: 
            epoch_lst.append(e)
            val_loss_lst.append(loss_v)
            train_loss_lst.append(loss_t)
            successes_count += 1
            print("successful")

        print("stop_epoch = ", stop_epoch)

        # Rand new weights and biases
        new_w1, new_w2, new_b1, new_b2 = rand_weights(in_dim, h_dim, out_dim, bridge)


    # Update previous training results
    results = dict()
    results["meanepochs"] = np.mean(epoch_lst)
    results["std/epochs%"] = 100 * np.std(epoch_lst) / prev_results["meanepochs"]
    results["Failures"] = num_failures
    results["meanvalidloss"] = np.mean(val_loss_lst)
    results["stdvalidlossPercent"] = 100 * np.std(val_loss_lst) / prev_results["meanvalidloss"]
    results["meanTrainLoss"] = np.mean(train_loss_lst)
    results["stdTrainLossPercent"] = 100 * np.std(train_loss_lst) / prev_results["meanTrainLoss"]
    results["hidden_node_outputs"] = h_node_lst
    return results



In [12]:
for i, expr_params in enumerate(experiments):
  print(f"\nExperiment-{i+1}, parameters: {expr_params}")
  open("output.txt", "a").write("\n\n")
  open("output.txt", "a").write("\nExperiment " + str(i+1) + ": "+str(expr_params))
  results = train_GD(x_train, t_train, x_val, t_val, expr_params, results)
  print(results)
  open("output.txt", "a").write("\n meanepocs: "+ str(results["meanepochs"]) +", std/epocs% "+ str(results["std/epochs%"]) + " , Failures: "+ str(results["Failures"]))
  open("output.txt", "a").write("\n meanvalidloss: "+ str(results["meanvalidloss"]) + ", stdvalidlossPercent: " + str(results["stdvalidlossPercent"]) + ", ")
  open("output.txt", "a").write("\n meanTrainLoss: "+ str(results["meanTrainLoss"]) +", stdTrainLossPercent: " + str(results["stdTrainLossPercent"]))


Experiment-1, parameters: {'LR': 0.1, 'bridge': True, 'hidden': 2}
Training Session NO'0...
successful
stop_epoch =  5515
Training Session NO'1...
successful
stop_epoch =  4125
Training Session NO'2...
successful
stop_epoch =  4073
Training Session NO'3...
successful
stop_epoch =  5360
Training Session NO'4...
successful
stop_epoch =  3087
Training Session NO'5...
successful
stop_epoch =  3344
Training Session NO'6...
successful
stop_epoch =  4849
Training Session NO'7...
successful
stop_epoch =  3062
Training Session NO'8...
successful
stop_epoch =  3909
Training Session NO'9...
successful
stop_epoch =  6278
{'meanepochs': 4359.2, 'std/epochs%': inf, 'Failures': 0, 'meanvalidloss': 0.04627549, 'stdvalidlossPercent': inf, 'meanTrainLoss': 0.011272371, 'stdTrainLossPercent': inf, 'hidden_node_outputs': [array([[6.3898265e-03, 1.2510416e-01],
       [4.6590296e-07, 7.0551008e-01],
       [9.8625863e-01, 1.4238358e-02],
       [5.1728785e-03, 1.9484296e-01],
       [9.6511674e-01, 1.8787

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:130: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:133: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:135: RuntimeWarning: divide by zero encountered in double_scalars


successful
stop_epoch =  16763
Training Session NO'1...
successful
stop_epoch =  11879
Training Session NO'2...
successful
stop_epoch =  10553
Training Session NO'3...
successful
stop_epoch =  13690
Training Session NO'4...
successful
stop_epoch =  13624
Training Session NO'5...
successful
stop_epoch =  17638
Training Session NO'6...
successful
stop_epoch =  11442
Training Session NO'7...
successful
stop_epoch =  15604
Training Session NO'8...
successful
stop_epoch =  10790
Training Session NO'9...
successful
stop_epoch =  15364
{'meanepochs': 13733.7, 'std/epochs%': 55.27731727522456, 'Failures': 0, 'meanvalidloss': 0.12796561, 'stdvalidlossPercent': 72.88968056788362, 'meanTrainLoss': 0.038392033, 'stdTrainLossPercent': 54.59206570994982, 'hidden_node_outputs': [array([[0.975312  , 0.9042367 ],
       [0.99999607, 0.05572444],
       [0.01721185, 0.9998174 ],
       [0.99107003, 0.9716061 ],
       [0.04033449, 0.9996967 ],
       [0.9788338 , 0.98271185],
       [0.9901079 , 0.96782